In [1]:
import projectpath

from mosmo.model import DbXref, Molecule, Reaction
from mosmo.knowledge import kb

KB = kb.configure_kb()

In [2]:
KB.schema

{'EC': EC: [ref.EC] (EC/KbEntry),
 'GO': GO: [ref.GO] (GO/KbEntry),
 'CHEBI': CHEBI: [ref.CHEBI] (CHEBI/Molecule),
 'RHEA': RHEA: [ref.RHEA] (RHEA/Reaction),
 'compounds': compounds: [kb.compounds] (CANON/Molecule),
 'reactions': reactions: [kb.reactions] (CANON/Reaction),
 'pathways': pathways: [kb.pathways] (CANON/Pathway)}

In [3]:
KB.canon

{mosmo.model.core.Molecule: compounds: [kb.compounds] (CANON/Molecule),
 mosmo.model.core.Reaction: reactions: [kb.reactions] (CANON/Reaction),
 mosmo.model.core.Pathway: pathways: [kb.pathways] (CANON/Pathway)}

In [4]:
kb.as_xref("KEGG:C00002")

KEGG:C00002

In [5]:
kb.as_xref("C00002")

C00002

In [6]:
KB.deref("KEGG:C00002")

In [7]:
KB.xref(KB.compounds, "KEGG:C00002")

[[atp] adenosine 5'-triphosphate, [atp.full] ATP (fully protonated)]

In [8]:
KB.deref("EC:1.6.1.1")

[1.6.1.1] NAD(P)+ transhydrogenase (Si-specific)

In [9]:
KB.xref(KB.reactions, "EC:5.3.1.9")

[[pgi] G6P <=> F6P]

In [10]:
KB.deref("CHEBI:57540")

[57540] NAD(1-)

In [11]:
KB("CHEBI:57540")

[57540] NAD(1-)

In [12]:
KB("nad.ox").__dict__

{'id': 'nad.ox',
 'db': Datasource(id='CANON', name=None, home=None, urlpat=None),
 'name': 'NAD+',
 'shorthand': 'NAD+',
 'description': None,
 'aka': ['NAD(1-)',
  'NAD anion',
  "adenosine 5'-{3-[1-(3-carbamoylpyridinio)-1,4-anhydro-D-ribitol-5-yl] diphosphate}",
  'NAD(+)'],
 'xrefs': {CHEBI:57540, KEGG:C00003, METACYC:NAD},
 'formula': 'C21H26N7O14P2',
 'mass': 662.4172,
 'charge': -1,
 'inchi': 'InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/p-1/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1',
 'variations': None,
 'canonical_form': None,
 'default_form': None}

In [13]:
KB("pgi")

[pgi] G6P <=> F6P

In [14]:
KB(KB("pgi"))

[pgi] G6P <=> F6P

In [15]:
try:
    KB.deref(KB("pgi"))
except TypeError as e:
    print(e)

[pgi] G6P <=> F6P cannot be converted to DbXref.


In [16]:
for entry in (KB("h2o"), KB("h+"), KB("Glc.D"), KB("pgi"), KB("citsyn")):
    print(entry)
    for xref in entry.xrefs:
        xref_entry = KB.deref(xref, type(entry))
        url = xref.url(type(entry))
        print(f"    {xref} {xref_entry} / {url}")
    print()

[h2o] water
    WIKI:Water None / https://en.wikipedia.org/wiki/Water
    CHEBI:15377 [15377] water / http://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:15377
    KEGG:C00001 None / https://www.genome.jp/entry/C00001
    CAS:7732-18-5 None / None
    METACYC:WATER None / https://metacyc.org/compound?id=WATER

[h+] proton
    CHEBI:15378 [15378] hydron / http://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:15378
    METACYC:PROTON None / https://metacyc.org/compound?id=PROTON
    KEGG:C00080 None / https://www.genome.jp/entry/C00080

[Glc.D] D-glucose
    CAS:50-99-7 None / None
    CHEBI:17634 [17634] D-glucose / http://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:17634

[pgi] G6P <=> F6P
    ECOCYC:PGLUCISOM-RXN None / https://ecocyc.org/ECOLI/NEW-IMAGE?object=PGLUCISOM-RXN
    GO:0004347 [0004347] glucose-6-phosphate isomerase activity / http://amigo.geneontology.org/amigo/term/GO:0004347
    EC:5.3.1.9 [5.3.1.9] glucose-6-phosphate isomerase / https://enzyme.expasy.org/EC/5.3.1.9

In [17]:
from mosmo.knowledge.thermo import Thermodynamics

thermo = Thermodynamics()

In [18]:
thermo._cc_compounds

{}

In [19]:
thermo.cc.RT

2.4776264999999995 <Unit('kilojoule / mole')>

In [20]:
thermo.reaction_delta_g(KB("pgi"))

-0.049859255775004385

In [21]:
thermo.formation_delta_g(KB("Fru.D.6P"))

-1301.4485115635407

In [22]:
thermo.formation_delta_g(KB("Glc.D.6P"))

-1301.3986523077656

In [23]:
thermo.formation_delta_g(KB("Fru.D.6P")) - thermo.formation_delta_g(KB("Glc.D.6P"))

-0.04985925577511807

In [24]:
thermo.reaction_delta_g(KB("pfk"))

-16.177938426490925

In [25]:
sum(count * thermo.formation_delta_g(reactant) for reactant, count in KB("pfk").stoichiometry.items())

-16.177938426490982

In [26]:
thermo._cc_compounds

{[Glc.D.6P] D-glucose 6-phosphate: Compound(id=152, inchi_key=NBSCHQHZLSJFNQ-GASJEMHNSA-L),
 [Fru.D.6P] D-fructose 6-phosphate: Compound(id=125172, inchi_key=BGWGXPAPYGQALX-VRPWFDPXSA-N
 ),
 [atp] adenosine 5'-triphosphate: Compound(id=6, inchi_key=ZKHQWZAMYRWXGA-KQYNXXCUSA-J),
 [adp] adenosine 5'-diphosphate: Compound(id=10, inchi_key=XTWYTFMLZFPYCI-KQYNXXCUSA-K),
 [Fru.D.bis16] D-fructose 1,6-bisphosphate: Compound(id=396, inchi_key=RNBGYGVWRKECFJ-VRPWFDPXSA-J),
 [h+] proton: Compound(id=4, inchi_key=GPRLSGONYQIRFK-UHFFFAOYSA-N)}

In [27]:
mal = KB("mal.L")
thermo.pkas(mal)

[5.13, 3.2]

In [28]:
thermo.pkas(KB("Fru.D.6P"))

[13.99, 12.73, 10.28, 6.25, 1.22]

In [29]:
nad = KB("nad.ox")
nadh = KB("nad.red")
proton = KB("h+")
electron = Molecule(id='e-', name='electron', shorthand='e-', charge=-1, mass=0.000548579903, xrefs=(DbXref("CHEBI","10545"),))

halfcell = Reaction("nad_red", name="NAD reduction", stoichiometry={nad: -1, proton: -1, electron: -2, nadh: +1})
print(halfcell)
thermo.reaction_delta_g(halfcell)

[nad_red] NAD+ + H+ + 2 e- <=> NADH


66.78389426787089